In [1]:
import pandas as pd
import dask.dataframe as dd
import numpy as np
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

In [2]:
ddf = dd.read_csv('gs://dataproc-staging-us-central1-881288578123-co6sg2sg/prediction_csv/*.csv')
df = ddf.compute()

In [3]:
df[['On_Time_Probability', 'Delay_Probability']] = df['probability'].str.split(',', 1, expand=True)
df['On_Time_Probability'] = df['On_Time_Probability'].str.replace(r'\[', '',regex=True).astype(float)
df['Delay_Probability'] = df['Delay_Probability'].str.replace(r',.*','',regex=True).astype(float)
df['prediction'] = df['prediction'].astype(str).str.replace(r'0.0','On Time',regex=True)
df['prediction'] = df['prediction'].astype(str).str.replace(r'1.0','Late',regex=True)
df = df.drop(['probability'],axis=1)
df = df.rename({'prediction':'Prediction'},axis=1)

In [4]:
df

,OP_UNIQUE_CARRIER,ORIGIN,DEST,DEP_TIME,ARR_TIME,Prediction,On_Time_Probability,Delay_Probability
0,Jetblue Airways,BOS,ATL,Morning,Morning,Late,0.339385,0.660615
1,Envoy Air,DFW,MGM,Evening,Evening,On Time,0.857543,0.142457
2,Allegiant Air,ICT,AZA,Morning,Afternoon,On Time,0.844423,0.155577
3,Skywest Airlines,SBA,LAX,Afternoon,Evening,On Time,0.847446,0.152554
4,Spirit Airlines,PHL,LAX,Evening,Evening,On Time,0.833479,0.166521
...,...,...,...,...,...,...,...,...
13055,Mesa Airlines,STS,PHX,Morning,Afternoon,On Time,0.834287,0.165713
13056,Republic Airlines,DCA,TPA,Morning,Afternoon,Late,0.331570,0.668430
13057,Republic Airlines,MSY,JFK,Evening,Evening,Late,0.319799,0.680201
13058,Republic Airlines,DEN,BNA,Evening,Night,On Time,0.846459,0.153541


In [23]:
def delay_or_not(data,carrier,origin,destination,departureTime):
    if carrier != 'all':
        data = data[data['OP_UNIQUE_CARRIER'] == carrier]
    if departureTime != 'all':
        data = data[data['DEP_TIME'] == departureTime]
    if origin != 'all':
        data = data[data['ORIGIN'] == origin]
    if destination != 'all':
        data = data[data['DEST'] == destination]
    if len(data) == 0 : 
        return 'No Flights for this route.'
    else :
        data_on_time = data[data['Prediction'] == 'On Time']
        data_delay = data[data['Prediction'] == 'Late']
        print('We have a record of '+str(len(data))+' similar flights .')
        if len(data_on_time) >= len(data_delay):
            return ('Your flight should be on time, at a probability of : '+str(data_on_time['On_Time_Probability'].mean()*100)+'%')
        else:
            return ('Sorry, we forecast a delay with a probability of : '+str(data_delay['Delay_Probability'].mean()*100)+'%')

In [27]:
delay_or_not(df,'all','BOS','ATL','Evening')

We have a record of 34 similar flights .


'Sorry, we forecast a delay with a probability of : 68.62446302245337%'

In [17]:
di = {
"DEN": "DENVER",
"LAX": "LOS ANGELES", 
"DFW": "DALLAS", 
"LAS": "LAS VEGAS",
"OAK": "OAKLAND", 
"DTW": "DETROIT", 
"SEA": "SEATTLE",
"BUR": "HOLLYWOOD",
"IAH": "HOUSTON",
"MCI": "KANSAS", 
"BWI": "BALTIMORE",
"LGA": "NEW YORK", 
"MCO": "ORLANDO", 
"FLL": "FORT LAUDERDALE",
"JAX": "JACKSONVILLE", 
"ORD": "O'HARE", 
"STT": "CHARLOTTE AMALIE", 
"RSW": "SOUTHWEST FLORIDA", 
"MYR": "MYRTLE BEACH", 
"PDX": "PORTLAND", 
"SAN": "SAN DIEGO", 
"PHX": "PHOENIX", 
"SNA": "SANTA ANA",
"SLC": "SALT LAKE CITY",
"BOI": "BOISE", 
"SJC": "SAN JOSE",
"SAT": "SAN ANTONIO",
"JFK": "NEW YORK", 
"GEG": "SPOKANE",
"TUS": "TUCSON",
"SFO": "SAN FRANCISCO", 
"RDU": "RALEIGH",
"HNL": "HONOLULU",
"ATL": "ATLANTA",
"MSY": "NEW ORLEANS",
"DCA": "WASHINGTON", 
"EWR": "NEW YORK",
"ANC": "ANCHORAGE",
"TWF": "TWIN FALLS",
"HIB": "HIGGINS",
"MLI": "MOLINE",
"BGM": "BINGHAMPTON",
"MQT": "MARQUETTE",
"ABR": "ABERDEEN",
"PLN": "PELLSTON",
"LWS": "LEWISTON",
"APN": "ALPENA",
"GTF": "GREAT FALLS",
"ESC": "ESCANABA",
"RHI": "RHINERLANDER",
"IMT": "IRON MOUNTAIN",
"BTM": "BUTTE",
"BJI": "BEMIDJI",
"ABE": "ALLENTOWN",
"WYS": "WEST YELLOWSTONE",
"SWF": "NEWBURGH",
"COD": "CODY",
"SCE": "STATE COLLEGE",
"LGB": "LONG BEACH",
"EAR": "KEARNEY",
"PBG": "PLATTSBURGH",
"VEL": "VERNAL",
"DVL": "DEVILS LAKE",
"PHF": "NEWPORT NEWS", 
"EWN": "NEW BERN",
"OAJ": "JACKSONVILLE", 
"GPT": "GULFPORT", 
"DAB": "DAYTONA BEACH", 
"TLH": "TALLAHSEEE", 
"JAN": "JACKSON", 
"JLN": "JPOLIN", 
"AMA": "AMARILLO", 
"CRP": "CORPUS CHRISTI", 
"BRO": "BRONWSVILLE",
"LCH": "LAKE CHARLES", 
"AEX": "ALEXANDRIA", 
"SFB": "SANFORD", 
"AZA": "MESA", 
"PGD": "PUERTA GORDA", 
"BLV": "BELLEVILLE", 
"PIE": "ST. PETTERSBURG", 
"PVU": "PROVO", 
"HTS": "HUNTINGTON",
'LCK':'COLUMBUS',
'BMI':'BLOOMINGTON',
'USA':'CONCORD',
'PSM':'PORTSMOUTH',
'BLI':'BELLINGHAM',
'IAG':'NIAGARA FALLS',
'SCK':'STOCKTON',
'RFD':'ROCKFORD',
'EVV':'EVANSVILLE',
'HGR':'HAGERSTOWN',
'GRI':'GRAND ISLAND',
'SMX':'SANTA MARIA',
'OWB': 'OWENSBORO',
'OGD':'OGDEN',
'ELM':'ELMIRA',
'TOL':'TOLEDO',
'STC':'ST CLOUD',
'DBQ':'DUBUQUE',
'CLL':'COLLEGE STATION',
'TYR':'TYLER',
'ALO':'WATERLOO',
'TXK':'TEXARKANA',
'CMI':'SAVOY',
'SJT':'SAN ANGELO',
'ABI':'ALBIENE',
'GCK':'GARDEN CITY',
'HRL':'HARLINGEN',
'ISP':'NEW YORK',
}